# Project_Notes_007

## New York City Airbnb Reload
In Project_Notes_006, stable dlmc xgboost with dicvectorizer was used to buld a model for predicting New York City Airbnb apartment prices. In this project, the same data will be used but the model applied will be sklearn.ensemble.GradientBoostingRegressor, sklearn.compose.ColumnTransformer and sklearn.pipeline. The result will then be compared with that of Project_Notes_006.

In [1]:
# Surpress Warnings.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Import required modules.
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [2]:
# Define path.
path = './data/Project_Notes_006_NYC_Airbnb.parquet'

In [3]:
# Import the dataset.
airbnb = pd.read_parquet(path)

In [4]:
# Print the shape and head of the dataframe.
print(f'Shape of dataset: {airbnb.shape}\n')
airbnb.head()

Shape of dataset: (102599, 26)



,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,None
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,None
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,None,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,None,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",None
3,1002755,None,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,None,None
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",None


In [5]:
# Check info of dataframe.
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   host id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host name                       102193 non-null  object 
 5   neighbourhood group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  object 
 10  country code                    102468 non-null  object 
 11  instant_bookable                102494 non-null  object 
 12  cancellation_pol

In [6]:
# List all features.
airbnb.columns

Index(['id', 'NAME', 'host id', 'host_identity_verified', 'host name',
       'neighbourhood group', 'neighbourhood', 'lat', 'long', 'country',
       'country code', 'instant_bookable', 'cancellation_policy', 'room type',
       'Construction year', 'price', 'service fee', 'minimum nights',
       'number of reviews', 'last review', 'reviews per month',
       'review rate number', 'calculated host listings count',
       'availability 365', 'house_rules', 'license'],
      dtype='object')

In [7]:
# Define required columns and new dataframe.
cols = ['host_identity_verified', 'neighbourhood group', 'lat', 'long', 'instant_bookable', 
        'cancellation_policy', 'room type', 'Construction year', 'service fee', 'price']

df = airbnb[cols]

In [8]:
# Print the shape and head of the new dataframe.
print(f'Shape of dataframe: {df.shape}\n')
airbnb.head()

Shape of dataframe: (102599, 10)



,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,None
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,None
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,None,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,None,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",None
3,1002755,None,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,None,None
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",None


In [9]:
# Replace spaces in column names with underscore.
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.columns

Index(['host_identity_verified', 'neighbourhood_group', 'lat', 'long',
       'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'service_fee', 'price'],
      dtype='object')

In [10]:
# Check for missing values.
df.isnull().sum()

host_identity_verified    289
neighbourhood_group        29
lat                         8
long                        8
instant_bookable          105
cancellation_policy        76
room_type                   0
construction_year         214
service_fee               273
price                     247
dtype: int64

In [11]:
# Remove data with null values.
df = df.dropna()
df.isnull().sum()

host_identity_verified    0
neighbourhood_group       0
lat                       0
long                      0
instant_bookable          0
cancellation_policy       0
room_type                 0
construction_year         0
service_fee               0
price                     0
dtype: int64

In [12]:
# Check data types.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101535 entries, 0 to 102598
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   host_identity_verified  101535 non-null  object 
 1   neighbourhood_group     101535 non-null  object 
 2   lat                     101535 non-null  float64
 3   long                    101535 non-null  float64
 4   instant_bookable        101535 non-null  object 
 5   cancellation_policy     101535 non-null  object 
 6   room_type               101535 non-null  object 
 7   construction_year       101535 non-null  float64
 8   service_fee             101535 non-null  object 
 9   price                   101535 non-null  object 
dtypes: float64(3), object(7)
memory usage: 8.5+ MB


In [13]:
# Remove ',' and '$' signs from price and service_fee columns.
df['price'] = df.price.str.replace(',', '')
df['price'] = df.price.str.replace('$', '')
df['service_fee'] = df.service_fee.str.replace(',', '')
df['service_fee'] = df.service_fee.str.replace('$', '')
df.head()

,host_identity_verified,neighbourhood_group,lat,long,instant_bookable,cancellation_policy,room_type,construction_year,service_fee,price
0,unconfirmed,Brooklyn,40.64749,-73.97237,False,strict,Private room,2020.0,193,966
1,verified,Manhattan,40.75362,-73.98377,False,moderate,Entire home/apt,2007.0,28,142
3,unconfirmed,Brooklyn,40.68514,-73.95976,True,moderate,Entire home/apt,2005.0,74,368
4,verified,Manhattan,40.79851,-73.94399,False,moderate,Entire home/apt,2009.0,41,204
5,verified,Manhattan,40.74767,-73.97500,True,flexible,Entire home/apt,2013.0,115,577


In [14]:
# Change the type of instant_bookable, construction_year, service_fee and price.
df['instant_bookable'] = df.instant_bookable.astype('int64')
df['construction_year'] = df.construction_year.astype('int64')
df['service_fee'] = df.service_fee.astype('int64')
df['price'] = df.price.astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101535 entries, 0 to 102598
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   host_identity_verified  101535 non-null  object 
 1   neighbourhood_group     101535 non-null  object 
 2   lat                     101535 non-null  float64
 3   long                    101535 non-null  float64
 4   instant_bookable        101535 non-null  int64  
 5   cancellation_policy     101535 non-null  object 
 6   room_type               101535 non-null  object 
 7   construction_year       101535 non-null  int64  
 8   service_fee             101535 non-null  int64  
 9   price                   101535 non-null  int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 8.5+ MB


In [15]:
# Check the head of the dataframe again.
df.head()

,host_identity_verified,neighbourhood_group,lat,long,instant_bookable,cancellation_policy,room_type,construction_year,service_fee,price
0,unconfirmed,Brooklyn,40.64749,-73.97237,0,strict,Private room,2020,193,966
1,verified,Manhattan,40.75362,-73.98377,0,moderate,Entire home/apt,2007,28,142
3,unconfirmed,Brooklyn,40.68514,-73.95976,1,moderate,Entire home/apt,2005,74,368
4,verified,Manhattan,40.79851,-73.94399,0,moderate,Entire home/apt,2009,41,204
5,verified,Manhattan,40.74767,-73.97500,1,flexible,Entire home/apt,2013,115,577


In [16]:
# Check for dimentionality
object = df.select_dtypes(include='object')

for col in object.columns:
    print(object[col].value_counts())
    print()

host_identity_verified
unconfirmed    50801
verified       50734
Name: count, dtype: int64

neighbourhood_group
Manhattan        43357
Brooklyn         41395
Queens           13152
Bronx             2687
Staten Island      943
brookln              1
Name: count, dtype: int64

cancellation_policy
moderate    34020
strict      33806
flexible    33709
Name: count, dtype: int64

room_type
Entire home/apt    53145
Private room       46068
Shared room         2208
Hotel room           114
Name: count, dtype: int64



In [17]:
# Convert 'brookln' in neighbourhood_group to 'Brooklyn'.
df.neighbourhood_group.replace('brookln', 'Brooklyn', inplace=True)

In [18]:
# Check dimentionality of neighbourhood_group.
df.neighbourhood_group.value_counts()

neighbourhood_group
Manhattan        43357
Brooklyn         41396
Queens           13152
Bronx             2687
Staten Island      943
Name: count, dtype: int64

In [19]:
# Define X and y.
df_use = df.copy()
y = df_use.price
X = df_use.drop('price', axis=1)

In [20]:
# Split dataset to train, test and validation sets.
df_use = df.copy()
y = df_use['price']
X = df_use.drop('price', axis=1)

X_full_train, X_test, y_full_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=1)

In [21]:
# Create the preprocessor.
numeric_features = ['instant_bookable', 'lat', 'long', "construction_year", "service_fee"]

categorical_features = ['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']

preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown="ignore"), categorical_features)
                                       )

In [22]:
# Train a decision tree regressor model.
dtr = make_pipeline(preprocessor, DecisionTreeRegressor())

dtr.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['instant_bookable', 'lat',
                                                   'long', 'construction_year',
                                                   'service_fee']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['host_identity_verified',
                                                   'neighbourhood_group',
                                                   'cancellation_policy',
                                                   'room_type'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

In [23]:
# Calculate the RMSE of dtr.
pred_val_dtr = dtr.predict(X_val)
RMSE_val_dtr = np.sqrt(mean_squared_error(y_val, pred_val_dtr))
print('RMSE_val_dtr:', RMSE_val_dtr)

RMSE_val_dtr: 1.700995742782619


This is better than the Project_Notes_006, RMSE_val_dtr: 1.705505988587087

In [24]:
# Train an xgb regressor model.
xgb = make_pipeline(preprocessor, XGBRegressor(eta=0.3, max_depth=6, min_child_weight=1))
xgb.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['instant_bookable', 'lat',
                                                   'long', 'construction_year',
                                                   'service_fee']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['host_identity_verified',
                                                   'neighbourhood_group',
                                                   'cancellation_policy',
                                                   'room_type'])])),
                ('xgbregressor',
                 XGBRegressor(base_...
                              eval_metric=None, feature_types=None, gamma=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None, min_child_weight=1,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None, ...))])

In [25]:
# Calculate the RMSE of xgb.
pred_val_xgb = xgb.predict(X_val)
RMSE_val_xgb = np.sqrt(mean_squared_error(y_val, pred_val_xgb))
print('RMSE_val_xgb:', RMSE_val_xgb)

RMSE_val_xgb: 1.3992452094562897


XGB is the better model, but less than the Project_Notes_006, RMSE_val_xgb: 1.383520541648735

In [26]:
# Use GridSearchCV to find the best hyperparameters.

params = {
            'xgbregressor__eta': [0.1, 0.2, 0.3],
            'xgbregressor__max_depth': [10, 20, 30],
            'xgbregressor__min_child_weight': [1, 3, 10]
         }

gscv = GridSearchCV(
                    estimator=xgb,
                    param_grid=params,
                    cv=5,
                    n_jobs=-1
                    )

gscv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         ['instant_bookable',
                                                                          'lat',
                                                                          'long',
                                                                          'construction_year',
                                                                          'service_fee']),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['host_identity_verified',
                                                                          'neighbourhood_group',
                                                                          'cancellation_policy',
                                                                          'room_type'])])),
                                       ('xgbr...
                                                     max_cat_threshold=None,
                                                     max_cat_to_onehot=None,
                                                     max_delta_step=None,
                                                     max_depth=6,
                                                     max_leaves=None,
                                                     min_child_weight=1,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None, ...))]),
             n_jobs=-1,
             param_grid={'xgbregressor__eta': [0.1, 0.2, 0.3],
                         'xgbregressor__max_depth': [10, 20, 30],
                         'xgbregressor__min_child_weight': [1, 3, 10]})

In [27]:
# Get model best parameters.
gscv.best_params_

{'xgbregressor__eta': 0.1,
 'xgbregressor__max_depth': 20,
 'xgbregressor__min_child_weight': 1}

The hyperparameters for Project_Notes_006 are eta = 0.3, max_depth = 20 and min_child_weight = 3

In [28]:
# Use the best parameters to fit the model.
xgb = make_pipeline(preprocessor, XGBRegressor(eta=0.1, max_depth=20, min_child_weight=1))
xgb.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['instant_bookable', 'lat',
                                                   'long', 'construction_year',
                                                   'service_fee']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['host_identity_verified',
                                                   'neighbourhood_group',
                                                   'cancellation_policy',
                                                   'room_type'])])),
                ('xgbregressor',
                 XGBRegressor(base_...
                              eval_metric=None, feature_types=None, gamma=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=20, max_leaves=None, min_child_weight=1,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None, ...))])

In [29]:
# Calculate the best RMSE of xgb.
pred_val_xgb = xgb.predict(X_val)
RMSE_val_xgb = np.sqrt(mean_squared_error(y_val, pred_val_xgb))
print('RMSE_val_xgb:', RMSE_val_xgb)

RMSE_val_xgb: 1.271811295945512


This is better than the Project_Notes_006, RMSE_test_xgb: 1.3121185139396214

In [30]:
# Training with full train.
final_model = make_pipeline(preprocessor, XGBRegressor(eta=0.1, max_depth=20, min_child_weight=1))
final_model.fit(X_full_train, y_full_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['instant_bookable', 'lat',
                                                   'long', 'construction_year',
                                                   'service_fee']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['host_identity_verified',
                                                   'neighbourhood_group',
                                                   'cancellation_policy',
                                                   'room_type'])])),
                ('xgbregressor',
                 XGBRegressor(base_...
                              eval_metric=None, feature_types=None, gamma=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=20, max_leaves=None, min_child_weight=1,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None, ...))])

In [31]:
# Calculate the RMSE of xgb with X_test.
pred_test_xgb = final_model.predict(X_test)
RMSE_test_xgb = np.sqrt(mean_squared_error(y_test, pred_test_xgb))
print('RMSE_test_xgb:', RMSE_test_xgb)

RMSE_test_xgb: 1.1899933093324253


This is better than the Project_Notes_006, RMSE_test_xgb: 1.2356099756582724

In [32]:
# Save the model to disk.
filename = './data/Project_Notes_007_model.pkl'
pickle.dump(final_model, open(filename, 'wb'))

In [33]:
# load the model from disk.
pickled_model = pickle.load(open(filename, 'rb'))

In [34]:
# Function to predict the raw data
def predict_data(data):
    data_frame = pd.DataFrame(raw_data, index=[0])
    data_predict = final_model.predict(data_frame)
    return data_predict[0]

In [35]:
# Select data to predict.
df.iloc[200, :]

host_identity_verified           verified
neighbourhood_group              Brooklyn
lat                               40.6848
long                            -73.96219
instant_bookable                        0
cancellation_policy              flexible
room_type                 Entire home/apt
construction_year                    2006
service_fee                           140
price                                 701
Name: 325, dtype: object

In [36]:
# Create dictionary for prediction.
raw_data = {
            'host_identity_verified': 'verified',
            'neighbourhood_group': 'Brooklyn',
            'lat': 40.6848,
            'long': -73.9621,
            'instant_bookable': 0,
            'cancellation_policy': 'flexible',
            'room_type': 'Entire home/apt',
            'construction_year': 2006,
            'service_fee': 140
            }

In [37]:
# Predict data.
predict_data(raw_data)

700.9817

In [38]:
# Select data to predict.
df.iloc[4002, :]

host_identity_verified        unconfirmed
neighbourhood_group             Manhattan
lat                              40.78587
long                            -73.94764
instant_bookable                        1
cancellation_policy              moderate
room_type                 Entire home/apt
construction_year                    2012
service_fee                           140
price                                 698
Name: 4293, dtype: object

In [39]:
# Create dictionary for prediction.
raw_data = {
            'host_identity_verified': 'unconfirmed',
            'neighbourhood_group': 'Manhattan',
            'lat': 40.78587,
            'long': -73.94764,
            'instant_bookable': 1,
            'cancellation_policy': 'moderate',
            'room_type': 'Entire home/apt',
            'construction_year': 2012,
            'service_fee': 140
            }

In [40]:
# Predict data.
predict_data(raw_data)

698.2185

In [41]:
# Select data to predict.
df.iloc[10202, :]

host_identity_verified        unconfirmed
neighbourhood_group                Queens
lat                              40.74267
long                            -73.78737
instant_bookable                        1
cancellation_policy              flexible
room_type                 Entire home/apt
construction_year                    2009
service_fee                           194
price                                 972
Name: 10557, dtype: object

In [42]:
# Create dictionary for prediction.
raw_data = {
            'host_identity_verified': 'unconfirmed',
            'neighbourhood_group': 'Queens',
            'lat': 40.74267,
            'long': -73.78737,
            'instant_bookable': 1,
            'cancellation_policy': 'flexible',
            'room_type': 'Entire home/apt',
            'construction_year': 2009,
            'service_fee': 194
            }

In [43]:
# Predict data.
predict_data(raw_data)

971.7138